# 02. Updating raw orthogroups with blast to produce inclusive orthogroups and using clustering to produce exclusive orthogroups (alleles)

In [ ]:
# blast to cluster orthogroups and non nlr orthogroup sequences
# clustering to split orthogroup sequences
# run genespace and take into account synteny
# compare to diamond deepclust (but not always appropriate as it is just a hard cut off)


In [ ]:
mamba activate dl20-orthogrouping
conda info --envs

In [ ]:
dl20=(
at6923 at6929 at7143
at8285 at9104 at9336 at9503
at9578 at9744 at9762 at9806
at9830 at9847 at9852 at9879
at9883 at9900
)

# Create the final NLR master table

### Start with the making the manual curated gene list, this is not based on the orthogrouping but is the gff entries labelled as manually curated (i.e. we looked at them and they were in the manually curated table)

In [ ]:
# only need to run this once

mkdir -p tmp/just_manually_annotated_final_actual

for accession in "${dl20[@]}"
do 
grep "manually_curated" ../final/annotation/latest/${accession}_hodgepodgemerged.gff3 \
   > tmp/just_manually_annotated_final_actual/${accession}.manual_annotated_genes.gff3
done

In [ ]:
# make a gff of all the final manually curated gffs

cat tmp/just_manually_annotated_final_actual/*.gff3 > tmp/just_manually_annotated_final_actual/all_manually_curated_genes.gff3

In [ ]:
cat tmp/just_manually_annotated_final_actual/all_manually_curated_genes.gff3 | wc -l

In [ ]:
# run python script Kevin wrote to make a table out of the 9th column in the gff with all the comments
python3 ./extractids.py \
    <tmp/just_manually_annotated_final_actual/all_manually_curated_genes.gff3 \
    > tmp/just_manually_annotated_final_actual/all_manually_curated_genes.attr.tsv

In [ ]:
cat tmp/just_manually_annotated_final_actual/all_manually_curated_genes.attr.tsv | wc -l

In [ ]:
head tmp/just_manually_annotated_final_actual/all_manually_curated_genes.attr.tsv

In [ ]:
# produce a gff of the extra NLR tracker sequence
# use this to add sequences that were not dealt with manually from nlr tracker

#extra_nlr_tracker_ids.txt

for id in $(cat tmp/just_manually_annotated_final_actual/nlr_extras.txt)
do 
grep ${id} ../final/annotation/latest/ALL_ACCESSIONS_hodgepodgemerged.gff3 \
   >> tmp/just_manually_annotated_final_actual/all.extra_nlr_tracker_genes.gff3
done

In [ ]:
python3 ./extractids.py \
    < tmp/just_manually_annotated_final_actual/all.extra_nlr_tracker_genes.gff3 \
    > tmp/just_manually_annotated_final_actual/all_extra_nlr_tracker_genes.attr.tsv

In [ ]:
#add the last few sequences identified through orthogrouping

In [ ]:
for id in $(cat tmp/just_manually_annotated_final_actual/few_extras_from_orthogrouping.txt)
do 
grep ${id} ../final/annotation/latest/ALL_ACCESSIONS_hodgepodgemerged.gff3 \
   >> tmp/just_manually_annotated_final_actual/all.extra_nlr_orthogrouping_genes.gff3
done

In [ ]:
python3 ./extractids.py \
    < tmp/just_manually_annotated_final_actual/all.extra_nlr_orthogrouping_genes.gff3 \
    > tmp/just_manually_annotated_final_actual/all_extra_nlr_orthogrouping_genes.attr.tsv

## Propduce a list of all the Orthogroups that contain manually curated sequences

In [ ]:
# The manually_curated_gene_ids.txt file is the subset of the all_manually_curated_genes.attr.tsv file that is gene or protopseudogene

In [ ]:
# Make a file of all orthogroups containing the manually annotated sequences

for accession in $(cat tmp/just_manually_annotated_final/all_manually_curated_and_nlrtracker_ids.txt)
do
grep ${accession} output/difflines_v3_aa_orthofinder/Orthogroups.txt >> tmp/just_manually_annotated_final/all_orthogroup_members_aa.txt
grep ${accession} output/difflines_v3_nt_orthofinder/Orthogroups.txt >> tmp/just_manually_annotated_final/all_orthogroup_members_nt.txt
done

In [ ]:
cat tmp/just_manually_annotated_final/all_manually_curated_and_nlrtracker_ids.txt | wc -l

In [ ]:
cat tmp/just_manually_annotated_final/all_orthogroup_members_aa.txt | wc -l
cat tmp/just_manually_annotated_final/all_orthogroup_members_nt.txt | wc -l

In [ ]:
sort tmp/just_manually_annotated_final/all_orthogroup_members_aa.txt | uniq > tmp/just_manually_annotated_final/all_orthogroups_containing_manually_curated_sequences_aa.txt
sort tmp/just_manually_annotated_final/all_orthogroup_members_nt.txt | uniq > tmp/just_manually_annotated_final/all_orthogroups_containing_manually_curated_sequences_nt.txt




In [ ]:
cat tmp/just_manually_annotated_final/all_orthogroups_containing_manually_curated_sequences_aa.txt | wc -l
cat tmp/just_manually_annotated_final/all_orthogroups_containing_manually_curated_sequences_nt.txt | wc -l

In [ ]:
sed 's/:.*//' tmp/all_orthogroups_containing_manually_curated_sequences.txt > tmp/Manually_curated_gene_orthogroups.txt

In [ ]:
cat tmp/Manually_curated_gene_orthogroups.txt | wc -l

In [ ]:
# put the sequence files for the manually curated orthogroups (obviously excluding at6137) in a folder

mkdir -p tmp/just_manually_annotated_final_actual/manually_curated_orthogroups

for orthogroup in $(cat tmp/just_manually_annotated_final_actual/manually_curated_orthogroups.txt)
do 
cp output/difflinesv4_orthofinder/OrthoFinder_final_difflines_aa/OrthoFinder/Results_Aug22/Orthogroup_Sequences/${orthogroup}.fa tmp/just_manually_annotated_final_actual/manually_curated_orthogroups/
done

Need to check whether these othogroups contain any sequence that wasn't manually annotated, they will at the very least because they won't contain at6137

In [ ]:
# make a list of all sequences in the orthogroups that contain manually annotated sequences
# run tmp/all_orthogroups_containing_manually_curated_sequences.txt through the reformat_orthofinder_results_notebook

In [ ]:
cat tmp/ortho-clean-map_just_orthogroups_containing_manually_annotated_sequences.tsv | wc -l

In [ ]:
head tmp/ortho-clean-map_just_orthogroups_containing_manually_annotated_sequences.tsv

In [ ]:
awk -F '\t' '{print $2}' tmp/ortho-clean-map_just_orthogroups_containing_manually_annotated_sequences.tsv > tmp/manual_orthogroups_seq_list.txt

In [ ]:
cut -d_ -f1,2 tmp/manual_orthogroups_seq_list.txt > tmp/manual_orthogroups_seq_list_gene.txt
tail -n +2 tmp/manual_orthogroups_seq_list_gene.txt > tmp/manual_orthogroups_seq_list_gene_no_header.txt

In [ ]:
head tmp/manual_orthogroups_seq_list_gene_no_header.txt
cat tmp/manual_orthogroups_seq_list_gene_no_header.txt |wc -l

In [ ]:
# Now export a table including all sequences in the orthogroups containing manually annotated sequences

In [ ]:
cat input/final_gffs/*_hodgepodgemerged.gff3 \
    | awk '$3 ~ /gene|pseudogene|pseudogenic_region/{print $0}' \
    >tmp/all_annotations.genes.gff3

In [ ]:
for sequence in $(cat tmp/manual_orthogroups_seq_list_gene_no_header.txt)
do
   grep ${sequence} tmp/all_annotations.genes.gff3 >> tmp/manual_orthogroups_seq_genes.gff3
done

In [ ]:
head tmp/manual_orthogroups_seq_genes.gff3

In [ ]:
# run python script Kevin wrote to make a table out of the 9th column in the gff with all the comments
python3 ./extractids.py \
    <tmp/manual_orthogroups_seq_genes.gff3 \
    > tmp/all_manually_curated_genes_plus_those_in_mc_orthogroups.attr.tsv

In [ ]:
head tmp/all_manually_curated_genes_plus_those_in_mc_orthogroups.attr.tsv

In [ ]:
# get the protein fasta files

In [ ]:
for orthogroup in $(cat tmp/Manually_curated_gene_orthogroups.txt)
do 
cp tmp/orthofinder/orthofinder_no_araport/orthofinder/Orthogroup_Sequences/${orthogroup}.fa tmp/manually_curated_gene_orthogroups/
done

In [ ]:
# cat the new table with the orthogroup list
tmp/ortho-clean-map_just_orthogroups_containing_manually_annotated_sequences.tsv

In [ ]:
## now pull out all the sequences that are nominally actually in nlr orthogroups

for accession in $(cat tmp/just_manually_annotated_final/actual_NLR_aa_orthogroups.txt)
do
grep ${accession} output/difflines_v3_aa_orthofinder/Orthogroups.txt >> tmp/just_manually_annotated_final/all_orthogroups_actually_NLRs_aa.txt
done


In [ ]:
#Assign all pseudogenes and pseudogeneic regions to orthogroups

In [ ]:
# Take all nlr proteins and map to all known pseudogenes and pseudogenic regions

In [ ]:
mkdir -p tmp/just_manually_annotated_final/aa_NLR_orthogroups

for orthogroup in $(cat tmp/just_manually_annotated_final/all_orthogroups_actually_NLRs_aa_ids.txt)
do
cp output/difflines_v3_aa_orthofinder/Orthogroup_Sequences/${orthogroup}.fa tmp/just_manually_annotated_final/aa_NLR_orthogroups/
done

In [ ]:
cat tmp/just_manually_annotated_final/aa_NLR_orthogroups/*.fa > tmp/just_manually_annotated_final/all_nlr_aa.fasta

In [ ]:
cat output/per_accession_aa_cds/*_cds_aa.fasta > tmp/just_manually_annotated_final_actual/all_proteins.fasta

In [ ]:
cat output/per_accession_nt_pseudogene/*_pseudogene_nt.fasta output/per_accession_nt_pseudogenic_region/*.fasta > tmp/just_manually_annotated_final_actual/all_pseudoegenes_andregions.fasta


In [ ]:
miniprot -t8 tmp/just_manually_annotated_final_actual/all_pseudoegenes_andregions.fasta tmp/just_manually_annotated_final_actual/all_proteins.fasta > tmp/just_manually_annotated_final_actual/output_test_nlr_aa_against_all_ppr.paf

In [ ]:
for pseudo in $(cat tmp/just_manually_annotated_final_actual/ppr_that_are_mc_or_nlrtracker.txt)
do
grep "${pseudo}" tmp/just_manually_annotated_final_actual/output_test_nlr_aa_against_all_ppr.paf >> tmp/just_manually_annotated_final_actual/subset_of_ppr_flagged2.paf
done

In [ ]:
cat tmp/just_manually_annotated_final_actual/manually_curated_orthogroups/*.fa > tmp/just_manually_annotated_final_actual/all_mc_proteins.fasta

In [ ]:
miniprot -t8 tmp/just_manually_annotated_final_actual/all_pseudoegenes_andregions.fasta \
    tmp/just_manually_annotated_final_actual/all_mc_proteins.fasta \
    > tmp/just_manually_annotated_final_actual/output_test_mc_aa_against_all_ppr.paf

In [ ]:
cat tmp/just_manually_annotated_final_actual/manually_curated_orthogroups/*.fa > tmp/just_manually_annotated_final_actual/all_mc_proteins.fasta

In [ ]:
miniprot -t8 tmp/just_manually_annotated_final_actual/all_pseudoegenes_andregions.fasta \
    tmp/just_manually_annotated_final_actual/all_mc_proteins.fasta \
    > tmp/just_manually_annotated_final_actual/output_test_mc_aa_against_all_ppr.paf

## Assign all pseudogenes and pseudogeneic regions to orthogroups

In [ ]:
cat output/per_accession_aa_cds/*_cds_aa.fasta > tmp/all_proteins.fasta

In [ ]:
cat tmp/all_proteins.fasta | wc -l

In [ ]:
head tmp/all_proteins.fasta

In [ ]:
cat output/per_accession_nt_pseudogene/at*_pseudogene_nt.fasta > output/per_accession_nt_pseudogene/all_pseudogenes_nt.fasta
cat output/per_accession_nt_pseudogenic_region/at*_pseudogenic_region_nt.fasta > output/per_accession_nt_pseudogenic_region/all_pseudogenic_regions.fasta

In [ ]:
cat output/per_accession_nt_pseudogene/all_pseudogenes_nt.fasta output/per_accession_nt_pseudogenic_region/all_pseudogenic_regions.fasta > all_pseudo_and_pseudogenic_regions.fasta

In [ ]:
#This blast will only assign pseudogenes to the manually curated orthogroups
makeblastdb -in all_proteins.fasta -dbtype prot -out all_proteins
blastx -db all_proteins \
    -query all_pseudo_and_pseudogenic_regions.fasta \
    -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen slen gaps ppos frames qframe sframe sallgi" \
    -out all_pseudo_and_pseudogenic_regions_all_proteins_aa.tsv -num_threads 16

In [ ]:
cat all_pseudo_and_pseudogenic_regions_all_proteins_aa.tsv | sort -k1,1 -k13,13nr -k12,12n | sort -u -k1,1 --merge > all_pseudo_and_pseudogenic_regions_all_proteins_aa_single_hit.tsv

In [ ]:
# Next based on the orthogroups assign pseudogenes to orthogroups based on the protein orthofinder
# probably write a python script to do this

In [ ]:
cat tmp/manually_curated_pseudo_gene_ids.txt | wc -l

In [ ]:
# for those with hits produce gff table and quiry the gene orthogroup table for the orthogroup ids, need to check that all manually curated pseudogenes are included

In [ ]:
for sequence in $(cat tmp/manually_curated_pseudo_gene_ids.txt)
do 
grep "${sequence}" tmp/all_pseudo_and_pseudogenic_regions_all_proteins_aa_single_hit.tsv >> tmp/blast_hits_pseudo_manually_curated4.txt || echo "${sequence}"
done



In [ ]:
head tmp/blast_hits_pseudo_manually_curated4.txt

In [ ]:
cat tmp/blast_hits_pseudo_manually_curated4.txt | wc -l

In [ ]:
cat tmp/manually_curated_pseudo_gene_ids.txt | wc -l

In [ ]:
awk -F '\t' '{print $2}' tmp/blast_hits_pseudo_manually_curated2.txt > tmp/manual_orthogroups_seq_list_pseudo.txt 


In [ ]:
head tmp/manual_orthogroups_seq_list_pseudo.txt

In [ ]:
cat tmp/manual_orthogroups_seq_list_pseudo.txt | wc -l

In [ ]:
cut -d_ -f1,2 tmp/manual_orthogroups_seq_list_pseudo.txt > tmp/manual_orthogroups_seq_list_pseudo_gene.txt
tail -n +2 tmp/manual_orthogroups_seq_list_pseudo_gene.txt > tmp/manual_orthogroups_seq_list_pseudo_gene_no_header.txt

In [ ]:
cat tmp/manual_orthogroups_seq_list_pseudo_gene.txt | wc -l

In [ ]:
for sequence in $(cat tmp/manual_orthogroups_seq_list_pseudo_gene.txt)
do 
   grep "${sequence}" tmp/ortho-clean-map_just_orthogroups_containing_manually_annotated_sequences.tsv >> orthogroups_for_the_pseudo.txt
done

In [ ]:
cat orthogroups_for_the_pseudo.txt | wc -l

In [ ]:
head orthogroups_for_the_pseudo.txt

In [ ]:
for sequence in $(cat tmp/manually_curated_pseudo_gene_ids.txt)
do 
grep "${sequence}" tmp/just_manually_annotated/all_manually_curated_genes.gff3 >> tmp/just_pseudo_manually_curated_gffs2.gff3
done

In [ ]:
cat tmp/just_pseudo_manually_curated_gffs2.gff3 | wc -l

In [ ]:
python3 ./extractids.py \
    <tmp/just_pseudo_manually_curated_gffs2.gff3 \
    > tmp/just_manually_annotated/just_pseudo_manually_curated_genes.attr.tsv

In [ ]:
# Then Pass Kevin's table to assign NLR or not

In [ ]:
# Then to confirm inclusivity, blast all non NLR sequences against the proteins to check if any other othogroup needs to be included
# and then blast orthogroups against themselves to check clustering 

In [ ]:
mkdir -p tmp/just_nucleotide_regions_all

cp output/per_accession_nt_gene/*_gene_nt.fasta tmp/just_nucleotide_regions_all/
cp output/per_accession_nt_pseudogene/*_pseudogene_nt.fasta tmp/just_nucleotide_regions_all/
cp output/per_accession_nt_pseudogenic_region/*_pseudogenic_region_nt.fasta tmp/just_nucleotide_regions_all/

In [ ]:
#Output_nucleotide_alignments with pseudogenes

python3 src/fasta_formatter_general_with_an_orthofinder_list_1.2.py -t tmp/all_orthogroup_list_for_nucleotides.txt tmp/just_nucleotide_regions_all/*.fasta

In [ ]:
ls tmp/first_try_nt_orthogroups_with_pseudogenes/* > tmp/per_ortholog_files_nt_names.txt

mkdir tmp/per_ortholog_files_nt_mafft

for sample in $(cat tmp/per_ortholog_files_nt_names.txt)
do
    mafft --auto ${sample} > ${sample}_mafft.fas
done

# Try nucleotide orthofinder

In [ ]:
mkdir -p output/all_nucleotide_genes_and_pseudo_per_accession


for accession in "${dl20[@]}"
do 
cat output/per_accession_nt_gene/${accession}_gene_nt.fasta \
    output/per_accession_nt_pseudogene/${accession}_pseudogene_nt.fasta \
    output/per_accession_nt_pseudogenic_region/${accession}_pseudogenic_region_nt.fasta \
   > output/all_nucleotide_genes_and_pseudo_per_accession/${accession}_gppr.fasta
done

In [ ]:
# Then run on cluster

In [ ]:
cat tmp/list_of_known_sequences.txt |wc -l

In [ ]:
#list_of_known_sequences.txt

for sequence in $(cat tmp/list_of_known_sequences.txt)
do
    grep "${sequence}" tmp/orthofinder/orthofinder_nt_gppr_araport/OrthoFinder_gppr_araport/OrthoFinder/Results_Jul11_1/Orthogroups/Orthogroups.txt  >> tmp/nt_orthofinder_subset3.txt
done

In [ ]:
cat tmp/nt_orthofinder_subset3.txt | wc -l

In [ ]:
sed 's/\|/ /' tmp/nt_orthofinder_subset3.txt | awk '{print $1}' > tmp/nt_orthofinder_subset3_ogs.txt

# Also try mmseqs for nucleotide clusterin

In [ ]:
mkdir -p tmp/all_nucleotide_genes_and_pseudo_mmseqs_test

cat tmp/all_nucleotide_genes_and_pseudo_per_accession/*.fasta > tmp/all_nucleotide_genes_and_pseudo_mmseqs_test/all_gppr_plus_araport.fasta

In [ ]:
cd tmp/all_nucleotide_genes_and_pseudo_mmseqs_test

In [ ]:
mmseqs easy-cluster all_gppr_plus_araport.fasta clusterRes tmp --min-seq-id 0.5 -c 0.8 --cov-mode 1

In [ ]:
cd ../../

In [ ]:
ls tmp/all_nucleotide_genes_and_pseudo_mmseqs_test

In [ ]:
python3 src/fasta_formatter_general_with_an_orthofinder_list_1.3.py \
    -t tmp/all_nucleotide_genes_and_pseudo_mmseqs_test/clusterRes_cluster.tsv \
    tmp/just_nucleotide_regions_all/*.fasta

In [ ]:
mkdir -p tmp/all_nucleotide_genes_and_pseudo_mmseqs_test/fasta_clusters

In [ ]:
mv *_nt.fa tmp/all_nucleotide_genes_and_pseudo_mmseqs_test/fasta_clusters/

## find which sequences fall into the neighbourhood bed files

In [ ]:
cat input/neigbourhoods_new/*.bed > input/neigbourhoods_new/all_neighbour_hoods.bed

In [ ]:
bedtools intersect -a tmp/just_manually_annotated_final/all_manually_curated_and_nlrtracker.bed \
   -b input/neigbourhoods_new/all_neighbour_hoods.bed -wb > tmp/just_manually_annotated_final/all_nlr_neighbourhoods.txt

In [ ]:
cat input/final_gffs/*_hodgepodgemerged.gff3 > tmp/all_accessions_hodgepodge.gff3

In [ ]:
#Extract just rpp1
bedtools intersect -a tmp/all_accessions_hodgepodge.gff3 -b input/neigbourhoods/chr3_nh8/chr3_nh8.bed > input/neigbourhoods/chr3_nh8/chr3_ch8_all_entries.gff3

In [ ]:
grep "gene" input/neigbourhoods/chr3_nh8/chr3_ch8_all_entries.gff3 |wc -l

In [ ]:
grep "gene" input/neigbourhoods/chr3_nh8/chr3_ch8_all_entries.gff3 > input/neigbourhoods/chr3_nh8/chr3_ch8_all_entries_just_genes.gff3

In [ ]:
grep "at9578_G304230" tmp/orthofinder/orthofinder_no_araport/orthofinder/Orthogroups/Orthogroups.txt

In [ ]:
grep "at9578_G004410" input/final_gffs/at9578_hodgepodgemerged.gff3

In [ ]:
pwd

In [ ]:
grep "AT1G61300" tmp/orthofinder/orthofinder_aa_with_araport/Orthofinder_results_aa_with_araport11/OrthoFinder/Results_Jul05/Orthogroups/Orthogroups.txt

In [ ]:
grep "AT1G61300" tmp/orthofinder/orthofinder_nt_gppr_araport/OrthoFinder_gppr_araport/OrthoFinder/Results_Jul11_1/Orthogroups/Orthogroups.txt

In [ ]:
cp tmp/orthofinder/orthofinder_nt_gppr_araport/OrthoFinder_gppr_araport/OrthoFinder/Results_Jul11_1/Orthogroup_Sequences/OG0000087.fa tmp/

In [ ]:
cp tmp/orthofinder/orthofinder_nt_gppr_araport/OrthoFinder_gppr_araport/OrthoFinder/Results_Jul11_1/Orthogroup_Sequences/OG0000428.fa tmp/

## Convert the final gffs into bed format

In [ ]:
mkdir -p output/final_gffs_bed

for accession in "${dl20[@]}"
do 
gff2bed < input/final_gffs/${accession}_hodgepodgemerged.gff3 \
   > output/final_gffs_bed/${accession}_hodgepodgemerged.bed
done

In [ ]:
head output/final_gffs_bed/at9900_hodgepodgemerged.bed

In [ ]:
for accession in "${dl20[@]}"
do 
     grep -P '\tgene\t' output/final_gffs_bed/${accession}_hodgepodgemerged.bed > output/final_gffs_bed/${accession}_hodgepodgemerged_gene.bed
done

In [ ]:
head output/final_gffs_bed/at9900_hodgepodgemerged_gene.bed

## Comparison of annalena at9762 to our at9762

In [ ]:
pwd

In [ ]:
mkdir -p tmp/annalena_comparison

cat input/annalena_alignments_from_paper_supps/annalena_alignments/* > tmp/annalena_comparison/all_annalena_NLRs.fasta

In [ ]:
makeblastdb -in at9762_cds_aa.fasta -dbtype prot -out at9762_cds_aa
blastp -db at9762_cds_aa -query tmp/annalena_comparison/at9762_cds_aa.fasta -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen slen gaps ppos frames qframe sframe sallgi" -out at9762aa_ours_verses_annalena_aa.tsv -num_threads 4


In [ ]:
cat all_pseudo_and_pseudogenic_regions_all_proteins_aa.tsv | sort -k1,1 -k13,13nr -k12,12n | sort -u -k1,1 --merge > all_pseudo_and_pseudogenic_regions_all_proteins_aa_single_hit.tsv

In [ ]:
mkdir -p tmp/at9762_comparison

mv at9762.protein.fasta tmp/at9762_comparison

In [ ]:
cd tmp/at9762_comparison

In [ ]:
makeblastdb -in at9762_nlrs_manually_annotated.fasta -dbtype prot -out at9762_cds_aa_nlr
blastp -db at9762_cds_aa_nlr -query at9762.protein.fasta -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen slen gaps ppos frames qframe sframe sallgi" \
     -out at9762aa_ours_verses_augustus_nlr.tsv -num_threads 4